# 他リポジトリから実験の入力データを用意する

[データガバナンス機能](http://dg02.dg.rcos.nii.ac.jp/)に公開されているリポジトリやあなたのリポジトリのデータをこの実験の入力データとして用意します。  
一括でファイルの実体をダウンロードするのではなく、必要なファイルのみファイルの実体をダウンロードするため、大容量データでも時間がかかりにくくなります。  
上から順番に実行してください。

※もし入力データとして利用したいファイルの実体が「git-annex」によってAWS S3準拠のオブジェクトストレージに保存されている場合、[こちら](util/operate_s3_annex.ipynb)を実行することで当該データを取得できます。

## 1. 入力データにするリポジトリのURLおよび実験名を入力する

以下のセルを実行して、入力データにしたいリポジトリのURLおよび実験名を入力してください。  
入力したURLの履歴をこのノートブックに保存したい場合は、実行後にセルの左側に表示されるピンマークをクリックしてください。  
※入力に誤りがある場合は、再度このセルを実行して下さい。

In [ ]:
from ipywidgets import Text, Button, Layout
from IPython.display import clear_output

def on_click_callback(clicked_button: Button) -> None:
    global input_repo
    input_repo = text.value
    clear_output()
    print("入力完了しました：", input_repo)

# テキストボックス
text = Text(
    description='URL：',
    placeholder='http://dg02.dg.rcos.nii.ac.jp/user/repository_title.git',
    layout=Layout(width='500px')
)
button = Button(description='入力完了')
button.on_click(on_click_callback)
text.on_submit(on_click_callback)
display(text, button)

In [ ]:
from ipywidgets import Text, Button
from IPython.display import clear_output

def on_click_callback(clicked_button: Button) -> None:
    global input_experiment
    input_experiment = text.value
    clear_output()
    print("入力完了しました：", input_experiment)

# テキストボックス
text = Text(
    description='実験名：'
)
button = Button(description='入力完了')
button.on_click(on_click_callback)
text.on_submit(on_click_callback)
display(text, button)

## 2. リポジトリのディレクトリ階層をダウンロードする

In [ ]:
# URLからリポジトリ名を抽出する
input_repo_title = input_repo.split('/')[-1].replace('.git', '')

# 実験パッケージパスを取得する
%store -r EXPERIMENT_TITLE
experiment_path = '/home/jovyan/experiments/' + EXPERIMENT_TITLE

# 入力データリポジトリをサブデータセットとしてクローンする
%cd ~/
!datalad clone -d . $input_repo \$experiment_path/input_data/$input_repo_title

## 3. ファイルの実体をダウンロードする

出力データ(output_dataフォルダ配下)全ての実体をダウンロードしたい場合は`3.1`を、  
特定のファイルを選んで実体をダウンロードしたい場合は`3.2`を実行してください。

### 3.1 出力データ(output_dataフォルダ配下)の実体をダウンロードする

指定したリポジトリの出力データ(output_dataフォルダ配下)全ての実体をダウンロードしたい場合に実行してください。  
ただし、指定した他リポジトリのデータセット構成がパラメータ用の(RCOS_for_parameters)だった場合は、3.2の処理で取得するようにしてください。

In [ ]:
%cd $experiment_path/input_data/$input_repo_title/experiments/$input_experiment/output_data
!datalad get . -r

### 3.2 特定のファイルを指定して実体をダウンロードする

出力データの特定のファイルや、出力データ以外のファイルをダウンロードしたい場合に、以下のセルの`"ファイルパス"`をダウンロードしたいファイルパスに書き換えてから実行してください。  
※複数ファイルの実体をダウンロードしたい場合は、`!datalad get "ファイルパス"`の処理を必要な数だけコピーしてください。

In [ ]:
%cd $experiment_path/input_data/$input_repo_title/experiments/$input_experiment

# (例)
# !datalad get 'sample_a.txt'
# !datalad get 'output_data/sample_b.txt'
# !datalad get '[parameter実験名]/output_data/sample_c.txt'

!datalad get 'ファイルパス'

## 4. 実行結果をデータガバナンス機能に同期する

ここまでの内容を保存し、データガバナンス機能に同期します。  
以下のセルを実行してください。

In [ ]:
from IPython.display import display, Javascript
display(Javascript('IPython.notebook.save_checkpoint();'))

以下を実行して、`リポジトリ側の変更と競合しました。競合を解決してください。`と表示された場合は、[こちらのFAQ](http://dg02.dg.rcos.nii.ac.jp/G-Node/Info/wiki/%E3%83%AF%E3%83%BC%E3%82%AF%E3%83%95%E3%83%AD%E3%83%BC#1-1%E5%90%8C%E6%9C%9F%E5%87%A6%E7%90%86%E3%82%92%E5%AE%9F%E8%A1%8C%E3%81%99%E3%82%8B%E3%81%A8%E3%80%81%E3%83%AA%E3%83%9D%E3%82%B8%E3%83%88%E3%83%AA%E5%81%B4%E3%81%AE%E5%A4%89%E6%9B%B4%E3%81%A8%E7%AB%B6%E5%90%88%E3%81%97%E3%81%BE%E3%81%97%E3%81%9F%E3%80%82%E7%AB%B6%E5%90%88%E3%82%92%E8%A7%A3%E6%B1%BA%E3%81%97%E3%81%A6%E3%81%8F%E3%81%A0%E3%81%95%E3%81%84%E3%80%82%E3%81%A8%E8%A1%A8%E7%A4%BA%E3%81%95%E3%82%8C%E3%82%8B)を参考に競合を解決してください。

In [ ]:
import papermill as pm
from colorama import Fore
from IPython.display import clear_output

%cd ~/
save_path = [ experiment_path + '/input_data/' + input_repo_title, '/home/jovyan/.gitmodules']
try:
    pm.execute_notebook(
        'WORKFLOWS/EX-WORKFLOWS/util/base_datalad_save_push.ipynb',
        '/home/jovyan/.local/push_log.ipynb',
        parameters = dict(SAVE_MESSAGE = EXPERIMENT_TITLE + '_入力データの準備', PATH = save_path, IS_RECURSIVE = False, UNLOCK = False, REMOVE_PUSH_PATH = True)
    )
finally:
    clear_output()
    %store -r DATALAD_MESSAGE
    %store -r DATALAD_ERROR
    print('\n' + DATALAD_MESSAGE + '\n')
    print(Fore.RED + DATALAD_ERROR)

## 5. 実験中ワークフロー機能トップページに遷移する

続けてワークフロー機能を実行する場合は、[こちら](../experiment.ipynb)からトップページに遷移できます。  